<a href="https://colab.research.google.com/github/kanchanraiii/SecureRag/blob/master/FAISS_Minor_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu sentence-transformers


In [ ]:
from google.colab import files

uploaded = files.upload()


In [ ]:
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
docs = []
with open("ecommerce_dataset.jsonl", "r") as f:
    for line in f:
        obj = json.loads(line)

        # Join all key-value pairs into one text string
        text = " | ".join([f"{k}: {v}" for k, v in obj.items()])

        docs.append(text)

print(f"Loaded {len(docs)} documents")
print("Example doc:", docs[0][:300], "...")


In [ ]:
embeddings = model.encode(docs)
embeddings = np.array(embeddings).astype("float32")

d = embeddings.shape[1]   # embedding dimension
index = faiss.IndexFlatL2(d)
index.add(embeddings)

print(f"Indexed {len(docs)} documents")


In [ ]:
def search(query, k=3):
    q_vec = model.encode([query]).astype("float32")
    distances, indices = index.search(q_vec, k)
    results = [docs[i] for i in indices[0]]
    return results


In [ ]:
query = "Which customer ordered electronics?"
retrieved = search(query, k=2)

print("Query:", query)
print("Retrieved docs:")
for doc in retrieved:
    print("-", doc[:300], "...\n")